In [ ]:
%pip install requests
%pip install faker


In [84]:
import requests
from pprint import pprint
import json
path = "/api/events"
base_url = "http://localhost:8002"

endpoint = f"{base_url}{path}"


In [85]:
data={"description" : "test description"}

In [ ]:
response = requests.post(endpoint, data=json.dumps(data))

print(response.json())


In [ ]:
response = requests.get(endpoint + "/2")

pprint(response.json())


In [57]:
import requests
from pprint import pprint
import json
path = "/api/events"
base_url = "http://localhost:8002"

endpoint = f"{base_url}{path}?page=2"


In [ ]:
response = requests.get(endpoint)

pprint(response.json())

In [77]:
import requests
from pprint import pprint
import json
path = "/api/events"
base_url = "http://localhost:8002"

endpoint = f"{base_url}{path}"
data = {"description": "updated description"}

In [ ]:
response = requests.put(endpoint + "/1", data=json.dumps(data))

pprint(response.json())

In [61]:
import requests
from pprint import pprint
import json
path = "/api/events"
base_url = "http://localhost:8002"

endpoint = f"{base_url}{path}"
data = {"description": "updated description"}

In [ ]:
response = requests.delete(endpoint + "/5")

pprint(response.json())

In [ ]:
from faker import Faker
import random
events = 1000
pages = [
        "/", "/about", "/pricing", "/contact", 
        "/blog", "/products", "/login", "/signup",
        "/dashboard", "/settings"
    ]
fake = Faker()
session_ids = [fake.uuid4() for _ in range(20)]
path = "/api/events/"
base_url = "http://localhost:8002" # 127.0.0.1
create_endpoint = f"{base_url}{path}"
referrers = [
    "https://google.com",
    "https://facebook.com",
    "https://twitter.com",
    "https://linkedin.com",
    "",  # direct traffic
    "https://github.com"
]

for i in range(events):
    page = random.choice(pages)
    user_agent = random.choice([
            fake.chrome,
            fake.firefox,
            fake.safari,
            fake.opera,
            fake.user_agent
        ])()
    payload = dict(
        page=random.choice(pages),
        user_agent=user_agent,
        ip_address=fake.ipv4(),
        referrer=random.choice(referrers),
        session_id=random.choice(session_ids),
        duration=random.randint(50, 5000)  # Between 5 and 300 seconds
    )
    response = requests.post(create_endpoint, json=payload) # HTTP POST
    if response.ok:
        data = response.json()
        print(data)

In [114]:
import sys
from pathlib import Path
src_path = Path("../src").resolve()
sys.path.append(str(src_path))
from api.events.models import EventModel
from api.db.session import Session
from api.db.session import engine
from timescaledb.hyperfunctions import time_bucket
from sqlmodel import select, func

In [128]:
with Session(engine) as session:
    bucket = time_bucket("1 minute", EventModel.time)
    query = (select(bucket, EventModel.page, func.count()).group_by(bucket, EventModel.page)).order_by(bucket,
                                                                                      EventModel.page)
    compiled_query = query.compile(compile_kwargs={"literal_binds": True})
    print(compiled_query)
    results = session.exec(query).fetchall()
    pprint(results)


[(datetime.datetime(2025, 4, 19, 8, 19, tzinfo=datetime.timezone.utc), '/', 32),
 (datetime.datetime(2025, 4, 19, 8, 19, tzinfo=datetime.timezone.utc), '/about', 37),
 (datetime.datetime(2025, 4, 19, 8, 19, tzinfo=datetime.timezone.utc), '/blog', 25),
 (datetime.datetime(2025, 4, 19, 8, 19, tzinfo=datetime.timezone.utc), '/contact', 35),
 (datetime.datetime(2025, 4, 19, 8, 19, tzinfo=datetime.timezone.utc), '/dashboard', 24),
 (datetime.datetime(2025, 4, 19, 8, 19, tzinfo=datetime.timezone.utc), '/login', 38),
 (datetime.datetime(2025, 4, 19, 8, 19, tzinfo=datetime.timezone.utc), '/pricing', 31),
 (datetime.datetime(2025, 4, 19, 8, 19, tzinfo=datetime.timezone.utc), '/products', 34),
 (datetime.datetime(2025, 4, 19, 8, 19, tzinfo=datetime.timezone.utc), '/settings', 44),
 (datetime.datetime(2025, 4, 19, 8, 19, tzinfo=datetime.timezone.utc), '/signup', 35),
 (datetime.datetime(2025, 4, 19, 8, 20, tzinfo=datetime.timezone.utc), '/', 74),
 (datetime.datetime(2025, 4, 19, 8, 20, tzinfo=dat